In [1]:
from keras.models import Sequential
import numpy as np

Using TensorFlow backend.


In [2]:
import scipy.io as sio
import numpy.matlib

In [3]:
data = sio.loadmat("preprocessed_ICU2.mat")

In [4]:
N = data['Patient_No']
x0_train = data['x0_train']
x1_train = data['x1_train']
x0_test = data['x0_test']
x1_test = data['x1_test']
max_length = data['max_length'][0][0]
max_length_test = data['max_length_test'][0][0]
dim = data['dim'][0][0]

# x_test_fix = data['x_test2']
# y_test_fix = data['y_test2']
# y_test_fix2 = np.matlib.repmat(y_test_fix,1,max_length)
# y_test_fix2 = y_test_fix2.reshape((-1, max_length,1))

x_train = np.concatenate((x0_train, x1_train), axis=0)
x_test = np.concatenate((x0_test, x1_test), axis=0)
y_train = np.concatenate((np.zeros(x0_train.shape[0]),np.ones(x1_train.shape[0])),axis=0)
y_test = np.concatenate((np.zeros(x0_test.shape[0]),np.ones(x1_test.shape[0])),axis=0)
y_test2 = np.matlib.repmat(y_test,max_length_test,1)
y_test2 = y_test2.reshape((-1, max_length_test,1))

In [5]:
from keras.layers import GRU, Dense, Input, Masking
from keras.models import Model

In [65]:
model = Sequential()
hidden_units1 = 50
model.add(Masking(mask_value=0., input_shape=(max_length, dim)))
model.add(GRU(hidden_units1, recurrent_dropout=0.5))
model.add(Dense(1, activation='sigmoid'))

In [80]:
c_weight = {0 : 1., 1: 5}

In [67]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [91]:
model.fit(x_train, y_train, epochs=10, batch_size=64, class_weight=c_weight)

Epoch 1/10
4874/4874 [==============================] - 24s - loss: 0.2948 - acc: 0.9454    
Epoch 2/10
4874/4874 [==============================] - 31s - loss: 0.3085 - acc: 0.9304    
Epoch 3/10
4874/4874 [==============================] - 25s - loss: 0.2874 - acc: 0.9446    
Epoch 4/10
4874/4874 [==============================] - 25s - loss: 0.3145 - acc: 0.9446    
Epoch 5/10
4874/4874 [==============================] - 25s - loss: 0.3082 - acc: 0.9436    
Epoch 6/10
4874/4874 [==============================] - 24s - loss: 0.2991 - acc: 0.9454    
Epoch 7/10
4874/4874 [==============================] - 24s - loss: 0.3011 - acc: 0.9413    
Epoch 8/10
4874/4874 [==============================] - 23s - loss: 0.2989 - acc: 0.9475    
Epoch 9/10
4874/4874 [==============================] - 25s - loss: 0.3106 - acc: 0.9413    
Epoch 10/10
4874/4874 [==============================] - 25s - loss: 0.2973 - acc: 0.9460    


In [9]:
# for i in range(len(weights)):
#         print(weights[i].shape)

In [10]:
# model.summary()

In [69]:
inputs = Input(shape=(max_length_test, dim))
l0 = Masking(mask_value=0., input_shape=(max_length_test, dim))
l1 = GRU(hidden_units1, return_sequences=True)
l2 = Dense(1, activation='sigmoid')
x0 = l0(inputs)
x1 = l1(x0)
x2 = l2(x1)

In [70]:
model2 = Model(inputs=inputs, outputs=x2)

In [92]:
model2.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
l1.set_weights(model.layers[1].get_weights())
l2.set_weights(model.layers[2].get_weights())

In [93]:
model2.evaluate(x_test,y_test2,batch_size=64)

1220/1220 [==============================] - 17s    


[0.59057874992245529, 0.73419277003554051]

In [84]:
x_test.shape

(1220, 941, 88)

In [94]:
risk = model2.predict(x_test, batch_size=64)

In [95]:
sio.savemat('risk_scores.mat', {'risk':risk, 'y_test':y_test})
# sio.savemat('risk_scores.mat', {'risk':risk, 'y_test':y_test, 'i0_train':i0_train, 'i1_train':i0_train,'i0_test':i0_train,'i1_test':i0_train,})

In [18]:
from keras.utils import plot_model 
import pydot

In [19]:
plot_model(model2)